# Loading the VGG16 network with pretained weights

In [ ]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
model = keras.applications.vgg16.VGG16(weights = 'imagenet')
model.summary()

# Preprocessing an input image for VGG16

In [ ]:
img_path = keras.utils.get_file(
    fname = 'elephant.jpg',
    origin = 'https://img-datasets.s3.amazonaws.com/elephant.jpg')

In [ ]:
def get_img_array(img_path, target_size):
  img = keras.utils.load_img(img_path, target_size = target_size)
  array = keras.utils.img_to_array(img)
  array = np.expand_dims(array,axis = 0)
  array = keras.applications.vgg16.preprocess_input(array)
  return array

In [ ]:
img_array = get_img_array(img_path,target_size = ((224,224)))

In [ ]:
preds = model.predict(img_array)
print(keras.applications.vgg16.decode_predictions(preds, top=3)[0])

In [ ]:
np.argmax(preds[0])

# Setting up a model that returns the last convolutional output

In [ ]:
last_conv_layer_name = 'block5_conv3'
classifier_layer_name = ['block5_pool','flatten', 'fc1','fc2','predictions']
last_conv_layer = model.get_layer(last_conv_layer_name)
last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

# Reapplying the classifier on top of the last convolutional output

In [ ]:
classifier_input = keras.Input(shape = last_conv_layer.output.shape[1:])
x = classifier_input
for layer_name in classifier_layer_name:
  x = model.get_layer(layer_name)(x)
classifier_model = keras.Model(classifier_input, x)

# Retrieving the gradients of the top predicted class








In [ ]:
with tf.GradientTape() as tape:
  last_conv_layer_output = last_conv_layer_model(img_array)
  tape.watch(last_conv_layer_output)
  preds = classifier_model(last_conv_layer_output)
  top_pred_index = tf.argmax(preds[0])
  top_class_channel = preds[:,top_pred_index]

grads = tape.gradient(top_class_channel, last_conv_layer_output)

# Gradient pooling and channel importance weighting

In [ ]:
pooled_grads = tf.reduce_mean(grads, axis = (0,1,2)).numpy()
last_conv_layer_output = last_conv_layer_output.numpy()[0]
for i in range(pooled_grads.shape[-1]):
  last_conv_layer_output[:,:,i] *= pooled_grads[i]

heatmap = np.mean(last_conv_layer_output, axis = -1)

# Heatmap post-processing

In [ ]:
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

# Superimposing the heatmap on teh original picture

In [ ]:
import matplotlib.cm as cm

img = keras.utils.load_img(img_path)
img = keras.utils.img_to_array(img)

heatmap = np.uint8(255 * heatmap)

jet = cm.get_cmap('jet')
jet_colors = jet(np.arange(256))[:,:3]
jet_heatmap = jet_colors[heatmap]

jet_heatmap = keras.utils.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.utils.img_to_array(jet_heatmap)

superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = keras.utils.array_to_img(superimposed_img)

save_path = 'elephant_cam_vgg16.jpg'
superimposed_img.save(save_path)
plt.imshow(superimposed_img)